## Logging Modules

Logging in Python is a crucial process for tracking events that occur when your software runs. Instead of relying only on `print()` statements, logging provides a structured, flexible, and robust way to record information about application flow, errors, and warnings for later analysis and debugging.

It is implemented using Python's powerful built-in logging module

### What is Logging and Why is it Essential?

Logging involves recording specific events, along with contextual data (like timestamps), into a designated output, such as a file, the console, or an external system.

### Why not just print()?

| Feature  | print() Statements | Python logging Module |
|----------|----------|----------|
| Control    | None. Output is always sent to standard output.   | Fine-grained control over output format, destination, and severity level.|
| Severity    | None. All messages are treated equally.  | Uses Log Levels (DEBUG, INFO, ERROR) to filter messages based on importance.   |
| Context     | Requires manual insertion of timestamps/details.  |  Automatically includes timestamps, module name, line number, and process ID. |
| Destination | Console only.   | Console, files, network sockets, or even external logging services.|



## ⚙️ How Python Logging Works (The Architecture)

The Python logging module operates through a system of four main components that work together hierarchically.

### 1. Loggers (The Reporter)

Role: The component that application code directly uses to emit log messages. You request a logger by name, typically using `logging.getLogger(__name__)`.

Log Levels: Every logger has a <b>threshold level</b> set. It ignores any message sent to it that is less severe than its threshold.

### 2. Log Records (The Message)

Role: An object created every time a logging function (e.g., `logger.error()`) is called. It contains all the event information: the message, the severity level, the timestamp, the line number, etc.

### 3. Handlers (The Deliverer)

Role: Determines where the log record goes. Handlers take the log record and dispatch it to the appropriate destination.
- StreamHandler: Sends output to the console (standard output/error).
- FileHandler: Sends output to a disk file.
- RotatingFileHandler: Writes to a file but automatically rolls over to a new file when the current one reaches a certain size.

### 4. Formatters (The Stylist)

Role: Specifies the final layout and structure of the log message before it is outputted by the Handler. A formatter uses a standard string format to include variables like `levelname`, `asctime`, `module`, and `message`.

## 🚦 Logging Levels (The Filter)
Python defines five standard logging levels, ordered by severity. When you configure a logger, you decide the minimum level it will process.

|  Level   | Purpose  | When to Use |
|----------|----------|-------------|
|DEBUG (10)| Detailed information, typically only of interest when diagnosing problems. | Development stage; showing variable values, function entry/exit.|
| INFO (20)| Confirmation that things are working as expected.| Start/end of a major task, successful configuration loading.|
|WARNING(30)|An indication that something unexpected happened, or a problem is imminent (but the software is still working).|Deprecated usage, performance issues, non-critical failures.|
|ERROR (40)|Serious problem; the software has failed to perform some function.| Failed database query, incorrect input that halts a process.|
|CRITICAL (50)|A severe error, indicating the application itself may be unable to continue running.|Out-of-memory errors, inability to load vital configurations.|


`logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')` 

if you want to see INFO or DEBUG, is development stage of an application

In [1]:
import logging

# Set the logger configuration to process INFO level messages and higher
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def divide(a, b):
    logger.info(f"Attempting division of {a} by {b}")
    try:
        result = a / b
        logger.debug(f"Result is {result}") # Will NOT show up (level is INFO)
        return result
    except ZeroDivisionError:
        logger.error("Attempted to divide by zero!")
        return None

divide(10, 2)    # Output: INFO - Attempting division...
divide(10, 0)    # Output: INFO - Attempting division... AND ERROR - Attempted to divide by zero!

2025-10-31 10:30:54,060 - INFO - Attempting division of 10 by 2
2025-10-31 10:30:54,061 - INFO - Attempting division of 10 by 0
2025-10-31 10:30:54,062 - ERROR - Attempted to divide by zero!


## Best Practices for Python Logging
. <b>Use `getLogger(__name__)`:</b> Always create your logger at the module level using `logger = logging.getLogger(__name__)`. This gives your logger the name of the current module, making it easy to identify the source of messages in a large application.

. <b>Avoid Catching Generic Exception:</b> When handling errors, catch specific exceptions `(except FileNotFoundError:)` and use `logger.exception()` within the except block.

. <b>Use logger.exception() for Tracebacks:</b> When an exception is caught, calling logger.exception("A critical operation failed") will automatically log the message and include the full traceback (stack trace), which is vital for debugging.

  Note: `logger.exception()` is just `logger.error()` with the stack trace automatically included.

. <b>Configure Logging Once (Usually at Startup):</b> The `logging.basicConfig()` function is simple, but for complex apps, use a configuration file (like JSON or YAML) or the logging.config module to set up Handlers, Formatters, and Loggers only once at application startup.

Use f-strings only for Simple Messages: For performance, avoid complex f-string formatting inside DEBUG messages. The interpolation (the formatting work) is done even if the message is thrown away. Instead, pass arguments directly:

Good: logger.debug("Value is %s", variable_name)

Less Good: logger.debug(f"Value is {variable_name}") (The work is wasted if the log level is higher than DEBUG).

In [8]:
import logging

# Set the logger configuration to process INFO level messages and higher
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s')
logger = logging.getLogger(__name__)

logger.setLevel(logging.DEBUG) # explicitly set logger level

# Debug the logging configuration
print(f"Root logger level: {logging.getLogger().level}")
print(f"Current logger level: {logger.level}")
print(f"Effective level: {logger.getEffectiveLevel()}")

logger.debug('I do not think this is shown')
logger.info('This will be shown because of level introduced in basic configuration')

logger.warning('Hi! this is a warning')

2025-11-03 11:08:47,660 DEBUG    [1654576996.py:14] I do not think this is shown
2025-11-03 11:08:47,662 INFO     [1654576996.py:15] This will be shown because of level introduced in basic configuration
2025-11-03 11:08:47,662 WARNING  [1654576996.py:17] Hi! this is a warning


Root logger level: 20
Current logger level: 10
Effective level: 10


### Log to a file
`import logging`

### Set the logger configuration to process INFO level messages and higher
```
logging.basicConfig(
  level=logging.DEBUG,
  format='%(asctime)s %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
  datefmt = '%d-%m-%Y %H:%M:%S', # to change the format of date and time 
  filename = 'logs.txt'
)
logger = logging.getLogger(__name__) # or any relevant name
```



In [18]:
import logging

logger.handlers = []

logging.basicConfig(
  level=logging.INFO,
  format='%(asctime)s %(levelname)-8s %(message)s',
  datefmt = '%d-%b-%Y %H:%M:%S', # to change the format of date and time 
  #filename = 'logs.txt'
)
logger = logging.getLogger(__name__) # or any relevant name

logger.info('This will be shown because of level introduced in basic configuration')

2025-11-03 11:27:47,273 INFO     [344598440.py:13] This will be shown because of level introduced in basic configuration
